## CHAPTER 10
# Data Aggregation and Group Operations

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## 10.1 How to Think About Group Operations

In [2]:
df = pd.DataFrame(
    {
        "key1": ["a", "a", None, "b", "b", "a", None],
        "key2": pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
        "data1": np.random.standard_normal(7),
        "data2": np.random.standard_normal(7),
    }
)
df

,key1,key2,data1,data2
0,a,1,0.091872,1.168371
1,a,2,0.663906,1.335668
2,None,1,-0.419634,0.868027
3,b,2,-0.156269,2.014443
4,b,1,0.629614,-1.541613
5,a,<NA>,-0.962450,0.248568
6,None,1,-1.540972,1.574722


In [3]:
grouped = df["data1"].groupby(df["key1"])
grouped

In [4]:
grouped.mean()

key1
a   -0.068890
b    0.236673
Name: data1, dtype: float64

In [5]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1       0.091872
      2       0.663906
b     1       0.629614
      2      -0.156269
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,1,2
key1,,
a,0.091872,0.663906
b,0.629614,-0.156269


In [7]:
state = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = np.array([2005, 2005, 2006, 2005, 2006, 2005, 2006])
df["data1"].groupby([state, years]).mean()

CA  2005   -0.149272
    2006   -0.419634
OH  2005   -0.032198
    2006   -0.455679
Name: data1, dtype: float64

In [8]:
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,-0.068890,0.917535
b,1.5,0.236673,0.236415


In [9]:
df.groupby("key2").mean(numeric_only=True)

,data1,data2
key2,,
1,-0.309780,0.517377
2,0.253819,1.675055


In [10]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1     0.091872  1.168371
     2     0.663906  1.335668
b    1     0.629614 -1.541613
     2    -0.156269  2.014443

In [11]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [12]:
df.groupby("key1", dropna=False).size()

key1
a      3
b      2
NaN    2
dtype: int64

In [13]:
df.groupby(["key1", "key2"], dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [14]:
df.groupby("key1").count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


### Iterating over Groups

In [15]:
for name , group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1  0.091872  1.168371
1    a     2  0.663906  1.335668
5    a  <NA> -0.962450  0.248568
b
  key1  key2     data1     data2
3    b     2 -0.156269  2.014443
4    b     1  0.629614 -1.541613


In [16]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', np.int64(1))
  key1  key2     data1     data2
0    a     1  0.091872  1.168371
('a', np.int64(2))
  key1  key2     data1     data2
1    a     2  0.663906  1.335668
('b', np.int64(1))
  key1  key2     data1     data2
4    b     1  0.629614 -1.541613
('b', np.int64(2))
  key1  key2     data1     data2
3    b     2 -0.156269  2.014443


In [17]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces["b"]

,key1,key2,data1,data2
3,b,2,-0.156269,2.014443
4,b,1,0.629614,-1.541613


In [18]:
pieces

{'a':   key1  key2     data1     data2
 0    a     1  0.091872  1.168371
 1    a     2  0.663906  1.335668
 5    a  <NA> -0.962450  0.248568,
 'b':   key1  key2     data1     data2
 3    b     2 -0.156269  2.014443
 4    b     1  0.629614 -1.541613}

In [19]:
grouped = df.groupby({"key1": "key", "key2": "key",
                      "data1": "data", "data2": "data"}, axis="columns")
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0  0.091872  1.168371
1  0.663906  1.335668
2 -0.419634  0.868027
3 -0.156269  2.014443
4  0.629614 -1.541613
5 -0.962450  0.248568
6 -1.540972  1.574722
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


C:\Users\sahor\AppData\Local\Temp\ipykernel_17860\695880334.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped = df.groupby({"key1": "key", "key2": "key",


### Selecting a Column or Subset of Columns

In [20]:
df

,key1,key2,data1,data2
0,a,1,0.091872,1.168371
1,a,2,0.663906,1.335668
2,None,1,-0.419634,0.868027
3,b,2,-0.156269,2.014443
4,b,1,0.629614,-1.541613
5,a,<NA>,-0.962450,0.248568
6,None,1,-1.540972,1.574722


In [21]:
df.groupby("key1")["data1"]
df.groupby("key1")[["data2"]]

In [22]:
df["data1"].groupby(df["key1"])
df[["data2"]].groupby(df["key1"])

In [23]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     1.168371
     2     1.335668
b    1    -1.541613
     2     2.014443

In [24]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped

In [25]:
s_grouped.mean()

key1  key2
a     1       1.168371
      2       1.335668
b     1      -1.541613
      2       2.014443
Name: data2, dtype: float64

### Grouping with Dictionaries and Series

In [26]:
people = pd.DataFrame(
    np.random.randn(5, 5),
    columns=["a", "b", "c", "d", "e"],
    index=["Joe", "Steve", "Wanda", "Jill", "Trey"],
)
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,-1.877966,-0.479295,-1.368838,1.586174,-0.423635
Steve,0.949775,0.681658,-0.403841,0.572103,-1.118243
Wanda,-1.204978,NaN,NaN,0.945276,1.199094
Jill,-1.018017,1.444417,-2.018139,0.939825,-0.398694
Trey,0.010547,0.284347,0.716858,0.351882,-0.410016


In [27]:
mapping = {"a": "red", "b": "red", "c": "blue",
            "d": "blue", "e": "red", "f" : "orange"}


In [28]:
by_column = people.groupby(mapping, axis="columns")
by_column.sum()

C:\Users\sahor\AppData\Local\Temp\ipykernel_17860\2628240624.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  by_column = people.groupby(mapping, axis="columns")


,blue,red
Joe,0.217336,-2.780896
Steve,0.168262,0.513190
Wanda,0.945276,-0.005883
Jill,-1.078314,0.027705
Trey,1.068739,-0.115122


In [29]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [30]:
people.groupby(map_series, axis="columns").count()

C:\Users\sahor\AppData\Local\Temp\ipykernel_17860\2727320543.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  people.groupby(map_series, axis="columns").count()


,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### Grouping with Functions

In [31]:
people

,a,b,c,d,e
Joe,-1.877966,-0.479295,-1.368838,1.586174,-0.423635
Steve,0.949775,0.681658,-0.403841,0.572103,-1.118243
Wanda,-1.204978,NaN,NaN,0.945276,1.199094
Jill,-1.018017,1.444417,-2.018139,0.939825,-0.398694
Trey,0.010547,0.284347,0.716858,0.351882,-0.410016


In [32]:
people.groupby(len).sum()  # group by length of the index value

,a,b,c,d,e
3,-1.877966,-0.479295,-1.368838,1.586174,-0.423635
4,-1.007470,1.728764,-1.301281,1.291707,-0.808710
5,-0.255202,0.681658,-0.403841,1.517379,0.080851


In [33]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,-1.877966,-0.479295,-1.368838,1.586174,-0.423635
4,two,-1.018017,0.284347,-2.018139,0.351882,-0.410016
5,one,-1.204978,0.681658,-0.403841,0.572103,-1.118243


### Grouping by Index Levels

In [35]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                     [1, 3, 5, 1, 3]],
                                    names=["cty", "tenor"])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.711541 -0.005846 -0.084968  0.380662 -0.505763
1     -1.745281 -0.655106  0.387296 -0.201200 -1.248077
2      0.332581  0.620237  0.845177 -0.151664 -0.884521
3      0.485850 -0.025303  1.704168 -1.004555  1.043403

In [37]:
hier_df.groupby(level="cty", axis="columns").count()

C:\Users\sahor\AppData\Local\Temp\ipykernel_17860\2795167867.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  hier_df.groupby(level="cty", axis="columns").count()


cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3
